In [ ]:
#imports
import sys
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import os
import logging

import torch
import monai
from monai.data import DataLoader
from monai.transforms import (
    AddChanneld,
    CenterSpatialCropd,
    Compose,
    Resized,
    RandSpatialCropd,
    ScaleIntensityd,
    ToTensord,
    LoadImaged,
    Identityd,
)

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [ ]:
#definitions of path
MODEL_DIR = os.path.join("./SEResNet/")
path_train_data=os.path.join("../../data/trainValid_DL.csv")
filename_predictions_for_platt_scaling=os.path.join("./SEResNet/predictions_for_platt_scaling.csv")

In [ ]:
#hyperparameters which were selected during hyperparameter tuning
lr=1e-4
optimizer="none"
strategy="adam"
epoch=84

In [ ]:
#definition of batch size, numbers should be devisors of 64
BATCH_SIZE=2

In [ ]:
#load ADNI training dataset
trainValidMerged=pd.read_csv(path_train_data,index_col="PTID")

In [ ]:
#load data augmentations
train_transforms = Compose(
        [
            LoadImaged(keys=["img"]),
            AddChanneld(keys=["img"]),
            ScaleIntensityd(keys=["img"]),
            Resized(keys=["img"],spatial_size=(256,256,256)),
            RandSpatialCropd(keys=["img"],roi_size=(224,224,224),random_size =False),
            ToTensord(keys=["img"]),
        ]
    )

valid_transforms = Compose(
        [
            LoadImaged(keys=["img"]),
            AddChanneld(keys=["img"]),
            ScaleIntensityd(keys=["img"]),
            Resized(keys=["img"],spatial_size=(256,256,256)),
            CenterSpatialCropd(keys=["img"],roi_size=(224,224,224)),
            ToTensord(keys=["img"]),
        ]
    )


In [ ]:
#extract predictions of model during CV for platt scaling
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=101)
cvIt=0
#initialize lists to store predictions and labels for validation data
pred_val=[]
label_val=[]
#cross validation
for trainCross, validCross in kf.split(trainValidMerged,trainValidMerged.DX):
    cvIt+=1
    #extract validation dataset for split
    valid=trainValidMerged.iloc[validCross]
    #transform data to pytorch format
    Y_valid=pd.get_dummies(valid.DX,drop_first=True).to_numpy().squeeze()
    Y_valid=Y_valid.tolist()
    validDSNew = [{"img": img, "label": label} for img, label in zip(valid.filename, Y_valid)]
    valid_ds = monai.data.Dataset(data=validDSNew, transform=valid_transforms)
    valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=8, pin_memory=torch.cuda.is_available())
    #choose cudo as the devive if it is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #load DL model using monai
    model = monai.networks.nets.SEResNet152(num_classes=2,spatial_dims=3, in_channels=1)
    #load models trained during hyperparameter tuning
    PATH=MODEL_DIR+"model_"+str(strategy)+"_"+str(lr)+"_"+optimizer+"_"+str(cvIt)+"_"+str(epoch)+"_polyak_averaged.pth"
    model.load_state_dict(torch.load(PATH))  # Choose whatever GPU device number you want
    model=model.to(device)
    #change model to evaluation model
    model.eval()
    with torch.no_grad():
        #iterate over validation data
        for val_data in valid_loader:
            #load inputs and labels for validation dataset
            inputs=val_data["img"].cuda()
            labels=val_data["label"].cuda()
            #predict output of model
            outputs = model(inputs)
            #compute predictions
            pred=torch.nn.functional.softmax(outputs)
            #store predictions
            pred_val.extend(pred.cpu().detach().numpy().tolist())
            label_val.extend(labels.cpu().detach().numpy().tolist())

In [ ]:
#flatten list of predictions
pred_val_second=[i[1] for i in pred_val]

In [ ]:
#save model predictions and labels
df=pd.DataFrame({"predictions":pred_val_second,"labels":label_val})
df.to_csv(filename_predictions_for_platt_scaling)